In [14]:
import glob
import os
import time 
import numpy as np


import cv2
from scipy import ndimage

import slicer
import Elastix
import JupyterNotebooksLib as slicernb

import sitkUtils as su
import SimpleITK as sitk


In [15]:
root_dir = 'E:\\Test_data_set'
mni_path = 'C:\\Users\\User\\AppData\\Local\\NA-MIC\\Slicer 4.11.20200930\\mni_icbm152_t1_tal_nlin_asym_09a.nii'

In [16]:
sub_path_list = sorted(glob.glob(root_dir)) 

In [17]:
sub_path_list

['E:\\Test_data_set']

In [18]:
def load_nii(path):
    """Load dicom files to sitk image"""
    reader = sitk.ImageFileReader()
    reader.SetImageIO("NiftiImageIO")
    reader.SetFileName(path)
    image = reader.Execute()
    
    return image

In [19]:
def registration_elastix(fixedVolumeNode, movingVolumeNode):
    """Ridgid registration Elastix"""
    
    #Set Parmeter
    elastixLogic = Elastix.ElastixLogic()
    parameterFilenames = elastixLogic.getRegistrationPresets()[0][Elastix.RegistrationPresets_ParameterFilenames]

    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
    outputTransformNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTransformNode")

    regi =  elastixLogic.registerVolumes(fixedVolumeNode, movingVolumeNode, parameterFilenames = parameterFilenames,
                                         outputVolumeNode =  outputVolumeNode, outputTransformNode = outputTransformNode )
    return outputVolumeNode, outputTransformNode
        
        
#     if (slicer.util.getNode("register_tmp") == None):
#         outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode").SetName("register_tmp")
#     else:5 -
#          outputVolumeNode = slicer.util.getNode("register_tmp")
    
    #Execute 
  

In [20]:
def write_nii(image, path):
    """Write medical image to nii"""
    sitk.WriteImage(image, path)

In [21]:
def rescaleIntentsity(image, min_intensity , max_intensity ):
    """Rescale Image intensity range of [min , max]"""
    return sitk.RescaleIntensity(image, min_intensity, max_intensity)

In [22]:
def TransformMarkups(transformNode):    
#     originalFidNode=slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")

    
    markupsNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsFiducialNode')
#     markupsNode.AddControlPoint(vtk.vtkVector3d(0, -1, -1)) #AC
#     markupsNode.AddControlPoint(vtk.vtkVector3d(0, -23, 0 )) #PC
    markupsNode.AddControlPoint(vtk.vtkVector3d(8, -73.0, 19.0)) #V1
    markupsNode.AddControlPoint(vtk.vtkVector3d(37, -13.0, 51.0 )) #M1
  #  markupsNode.AddControlPoint(vtk.vtkVector3d(42, -20.0, 48.0 )) #S1
    markupsNode.AddControlPoint(vtk.vtkVector3d(5.0, 14.0, 47))  #Dosal_ACC +  up5
    markupsNode.AddControlPoint(vtk.vtkVector3d(12, -15, 18.0))  #Thalamus
#     markupsNode.AddControlPoint(vtk.vtkVect|or3d(5.0, 14.0, 57.0))  #Dorsal_ACC +  up15
#     markupsNode.AddControlPoint(vtk.vtkVector3d(5.0, -10.0, 51.0)) #Caudal_ACC
#     markupsNode.AddControlPoint(vtk.vtkVector3d(5.0, -10.0, 56.0)) #Caudal_ACC + up5
#     markupsNode.AddControlPoint(vtk.vtkVector3d(5.0, -10.0, 61.0)) #Caudal_ACC + up15
    
#     markupsNode.AddControlPoint(vtk.vtkVector3d(8, -73.0 - 5, 19.0)) #V1 _5
#     markupsNode.AddControlPoint(vtk.vtkVector3d(37, -13.0, 51.0 + 5)) #M1 _ 5
#     markupsNode.AddControlPoint(vtk.vtkVector3d(42, -20.0, 48.0 + 5)) #S1 _ 5
    
    markupsNode.SetAndObserveTransformNodeID(transformNode.GetID())
#    slicer.modules.transforms.logic().CreateDisplacementVolumeFromTransform(transformNode, markupsNode, False)
    narray = slicer.util.arrayFromMarkupsControlPoints(markupsNode, world=True)
    
    print(narray)
    slicer.mrmlScene.RemoveNode(markupsNode)
    return narray

In [23]:
scene = slicer.mrmlScene
sub_path_list = sorted(glob.glob(os.path.join(root_dir, 'S*')))
print(len(sub_path_list))
i=0

19


In [24]:
sub_path_list

['E:\\Test_data_set\\S30', 'E:\\Test_data_set\\S31', 'E:\\Test_data_set\\S32', 'E:\\Test_data_set\\S33', 'E:\\Test_data_set\\S35', 'E:\\Test_data_set\\S36', 'E:\\Test_data_set\\S38', 'E:\\Test_data_set\\S39', 'E:\\Test_data_set\\S42', 'E:\\Test_data_set\\S43', 'E:\\Test_data_set\\S44', 'E:\\Test_data_set\\S46', 'E:\\Test_data_set\\S47', 'E:\\Test_data_set\\S51', 'E:\\Test_data_set\\S54', 'E:\\Test_data_set\\S55', 'E:\\Test_data_set\\S56', 'E:\\Test_data_set\\S57', 'E:\\Test_data_set\\S58']

In [25]:
scene = slicer.mrmlScene
sub_path_list = sorted(glob.glob(os.path.join(root_dir, 'S*')))
print(len(sub_path_list))
i=0

mni_image = load_nii(mni_path)
MNI_node = su.PushVolumeToSlicer(mni_image)

for sub_path in sub_path_list:
    try:
        sub_name = sub_path.split("\\")[-1].split("_")[0]
       
        print(i)
        i+=1

        T1_path = glob.glob(os.path.join(sub_path, '*crop_T1.nii'))[0]
        T1_image = load_nii(T1_path)
        print('load dicom', T1_path) 

        #sitk image to VolumeNode
        T1_node = su.PushVolumeToSlicer(T1_image)
        print('push image to volumenode')

        MNI_registeredNode, MNI_transformNode = registration_elastix(T1_node , MNI_node)
        print('mni-registration')
        
        target_np = TransformMarkups(MNI_transformNode)

        sub_name = sub_path.split("\\")[-1].split("_")[0]
        save_path = os.path.join(sub_path)

        if not os.path.exists(save_path):
            os.makedirs(save_path)
            print(save_path)
            
            
        V1 = str('{:.3f}'.format(target_np[0][0]) + ' {:.3f}'.format(target_np[0][1]) + ' {:.3f}'.format(target_np[0][2]))
        f = open(os.path.join(save_path, 'target_V1_MNI.txt'), 'w')
        f.write(V1)
        f.close()
               
        M1 = str('{:.3f}'.format(target_np[1][0]) + ' {:.3f}'.format(target_np[1][1]) + ' {:.3f}'.format(target_np[1][2]))
        f = open(os.path.join(save_path, 'target_M1_MNI.txt'), 'w')
        f.write(M1)

        
        Dorsal_ACC = str('{:.3f}'.format(target_np[2][0]) + ' {:.3f}'.format(target_np[2][1]) + ' {:.3f}'.format(target_np[2][2]))
        f = open(os.path.join(save_path, 'target_DorsalACCUp5.txt'), 'w')
        f.write(Dorsal_ACC)
        f.close()

        Thalamus = str('{:.3f}'.format(target_np[3][0]) + ' {:.3f}'.format(target_np[3][1]) + ' {:.3f}'.format(target_np[3][2]))
        f = open(os.path.join(save_path, 'target_Thalamus.txt'), 'w')
        f.write(Thalamus)
        f.close()

        
        print('save target mni to native space')

        #clear

        slicer.mrmlScene.RemoveNode(T1_node)
        slicer.mrmlScene.RemoveNode(MNI_transformNode)
        slicer.mrmlScene.RemoveNode(MNI_registeredNode)
        
        
    except Exception as e: 
        print(e)
        print(sub_path, 'wrong')
        continue

19
0
load dicom E:\Test_data_set\S30\S30_crop_T1.nii
push image to volumenode
Volume registration is started in working directory: C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957
Register volumes...


toVTKString is deprecated! Conversion is no longer necessary.


Register volumes using: C:\Users\User\AppData\Roaming\NA-MIC\Extensions-29402\SlicerElastix\lib\Slicer-4.11\elastix.exe: ['-f', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957\\input\\fixed.mha', '-m', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957\\input\\moving.mha', '-out', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957\\result-transform', '-p', 'C:\\Users\\User\\AppData\\Roaming\\NA-MIC\\Extensions-29402\\SlicerElastix\\lib\\Slicer-4.11\\qt-scripted-modules\\Resources\\RegistrationParameters\\Parameters_Rigid.txt', '-p', 'C:\\Users\\User\\AppData\\Roaming\\NA-MIC\\Extensions-29402\\SlicerElastix\\lib\\Slicer-4.11\\qt-scripted-modules\\Resources\\RegistrationParameters\\Parameters_BSpline.txt']
Show volume rendering of node Node
Show volume rendering of node Node_1
Show volume rendering of node Volume


Traceback (most recent call last):
  File "C:/Users/User/AppData/Roaming/NA-MIC/Extensions-29402/NvidiaAIAssistedAnnotation/lib/Slicer-4.11/qt-scripted-modules/.slicerrc.py", line 13, in <lambda>
    qt.QTimer.singleShot(10, lambda: showVolumeRendering(node))
  File "C:/Users/User/AppData/Roaming/NA-MIC/Extensions-29402/NvidiaAIAssistedAnnotation/lib/Slicer-4.11/qt-scripted-modules/.slicerrc.py", line 20, in showVolumeRendering
    scalarRange = volumeNode.GetImageData().GetScalarRange()
AttributeError: 'NoneType' object has no attribute 'GetScalarRange'


Generate output...
Generate output using: C:\Users\User\AppData\Roaming\NA-MIC\Extensions-29402\SlicerElastix\lib\Slicer-4.11\transformix.exe: ['-tp', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957\\result-transform/TransformParameters.1.txt', '-out', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957\\result-resample', '-in', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_182951_957\\input\\moving.mha', '-def', 'all']


...

Register volumes using: C:\Users\User\AppData\Roaming\NA-MIC\Extensions-29402\SlicerElastix\lib\Slicer-4.11\elastix.exe: ['-f', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_183953_594\\input\\fixed.mha', '-m', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_183953_594\\input\\moving.mha', '-out', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_183953_594\\result-transform', '-p', 'C:\\Users\\User\\AppData\\Roaming\\NA-MIC\\Extensions-29402\\SlicerElastix\\lib\\Slicer-4.11\\qt-scripted-modules\\Resources\\RegistrationParameters\\Parameters_Rigid.txt', '-p', 'C:\\Users\\User\\AppData\\Roaming\\NA-MIC\\Extensions-29402\\SlicerElastix\\lib\\Slicer-4.11\\qt-scripted-modules\\Resources\\RegistrationParameters\\Parameters_BSpline.txt']
Show volume rendering of node result_17
Show volume rendering of node Node_19
Show volume rendering of node Volume_18


Traceback (most recent call last):
  File "C:/Users/User/AppData/Roaming/NA-MIC/Extensions-29402/NvidiaAIAssistedAnnotation/lib/Slicer-4.11/qt-scripted-modules/.slicerrc.py", line 13, in <lambda>
    qt.QTimer.singleShot(10, lambda: showVolumeRendering(node))
  File "C:/Users/User/AppData/Roaming/NA-MIC/Extensions-29402/NvidiaAIAssistedAnnotation/lib/Slicer-4.11/qt-scripted-modules/.slicerrc.py", line 20, in showVolumeRendering
    scalarRange = volumeNode.GetImageData().GetScalarRange()
AttributeError: 'NoneType' object has no attribute 'GetScalarRange'


Generate output...
Generate output using: C:\Users\User\AppData\Roaming\NA-MIC\Extensions-29402\SlicerElastix\lib\Slicer-4.11\transformix.exe: ['-tp', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_183953_594\\result-transform/TransformParameters.1.txt', '-out', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_183953_594\\result-resample', '-in', 'C:/Users/User/AppData/Local/Temp/Slicer/Elastix/20211206_183953_594\\input\\moving.mha', '-def', 'all']


loadNodeFromFile `returnNode` argument is deprecated. Loaded node is now returned directly if `returnNode` is not specified.


Registration is completed
mni-registration
[[  8.82149579 -43.20634308  24.26274907]
 [ 38.34328711   3.96805258  62.55022157]
 [  5.20734297  31.66730641  59.83747671]
 [ 14.39446405   7.37188521  30.36956673]]
save target mni to native space
Show volume rendering of node result_18


In [13]:
scene = slicer.mrmlScene
sub_path_list = sorted(glob.glob(os.path.join(root_dir, 'S*')))
print(len(sub_path_list))
i=0

mni_image = load_nii(mni_path)
MNI_node = su.PushVolumeToSlicer(mni_image)

for sub_path in sub_path_list:
    try:
        sub_name = sub_path.split("\\")[-1].split("_")[0]
       
        print(i)
        i+=1

        T1_path = glob.glob(os.path.join(sub_path, '*crop_T1.nii'))[0]
        T1_image = load_nii(T1_path)
        print('load dicom', T1_path) 

        #sitk image to VolumeNode
        T1_node = su.PushVolumeToSlicer(T1_image)
        print('push image to volumenode')

        MNI_registeredNode, MNI_transformNode = registration_elastix(T1_node , MNI_node)
        print('mni-registration')
        
        target_np = TransformMarkups(MNI_transformNode)

        sub_name = sub_path.split("\\")[-1].split("_")[0]
        save_path = os.path.join(sub_path)

        if not os.path.exists(save_path):
            os.makedirs(save_path)
            print(save_path)
            
            
        V1 = str('{:.3f}'.format(target_np[0][0]) + ' {:.3f}'.format(target_np[0][1]) + ' {:.3f}'.format(target_np[0][2]))
        f = open(os.path.join(save_path, 'target_V1_MNI.txt'), 'w')
        f.write(V1)
        f.close()
               
        M1 = str('{:.3f}'.format(target_np[1][0]) + ' {:.3f}'.format(target_np[1][1]) + ' {:.3f}'.format(target_np[1][2]))
        f = open(os.path.join(save_path, 'target_M1_MNI.txt'), 'w')
        f.write(M1)

        
        Dorsal_ACC = str('{:.3f}'.format(target_np[2][0]) + ' {:.3f}'.format(target_np[2][1]) + ' {:.3f}'.format(target_np[2][2]))
        f = open(os.path.join(save_path, 'target_DorsalACCUp5.txt'), 'w')
        f.write(Dorsal_ACC)
        f.close()
        
        print('save target mni to native space')

        #clear

        slicer.mrmlScene.RemoveNode(T1_node)
        slicer.mrmlScene.RemoveNode(MNI_transformNode)
        slicer.mrmlScene.RemoveNode(MNI_registeredNode)
        
        
    except Exception as e: 
        print(e)
        print(sub_path, 'wrong')
        continue

19


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\D\S\Slicer-0-build\ITK\Modules\IO\NIFTI\src\itkNiftiImageIO.cxx:1025:
itk::ERROR: itk::ERROR: NiftiImageIO(000001B76805B2B0): C:\Users\User\AppData\Local\NA-MIC\Slicer 4.11.20200930\mni_icbm152_t1_tal_nlin_asym_09a.nii is not recognized as a NIFTI file

Show volume rendering of node mni_icbm152_t1_tal_nlin_asym_09a


In [ ]:
mni_icbm152_t1_tal_nlin_asym_09a.nii